In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.api import tsa
from dateutil.parser import parse

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def parse_quarter(string):
    """
    Converts a string from the format YYYYQN in datetime object at the end of quarter N.
    """
    
    # Note: you could also just retrieve the first four elements of the string
    # and the last one... Regex is fun but often not necessary
    year, qn = re.search(r'^(20[0-9][0-9])(Q[1-4])$', string).group(1, 2)
    
    # year and qn will be strings, pd.datetime expects integers.
    year = int(year)
    
    date = None
    
    if qn=='Q1':
        date = pd.datetime(year, 3, 31)
    elif qn=='Q2':
        date = pd.datetime(year, 6, 30)
    elif qn=='Q3':
        date = pd.datetime(year, 9, 20)
    else:
        date = pd.datetime(year, 12, 31)
        
    return date


alcohol_consumption = pd.read_csv('data/NZAlcoholConsumption.csv', 
                                  parse_dates=['DATE'], 
                                  date_parser=parse_quarter,
                                  index_col='DATE')
alcohol_consumption.sort_index(inplace=True)

In [ ]:
wine = alcohol_consumption.TotalWine
wine_diff = wine.diff(4).dropna()
time_series = wine_diff


## ARMA model

AR models a point in the time series as a linear model of the previous values. The mismatch $e_t$ is assumed to be "noise".
However there could still be information in the series of $e_t$! How about we add the past errors as additionnal features?

This leads to the **ARMA** model with an autoregressive part that you will recognise and a part that corresponds to a moving average:

$$
y_{t} = c + \underbrace{ \phi_{1}y_{t-1} + \phi_{2}y_{t-2} + \dots + \phi_{p}y_{t-p} }_{AR(p)} + \underbrace{ \theta_{1}e_{t-1} + \theta_{2}e_{t-2} + \dots + \theta_{q}e_{t-q} }_{MA(q)} +e_{t},
$$

ARMA models are also implemented in `statsmodels` and their implementation is consistent with the one of AR models. 

* Create an ARMA model with `tsa.ARMA`, specify $p=3$ and $q=3$
* Fit and predict
* Display

Since the result will look almost identical to just using AR, you will want to show the MAE as well. 

In [ ]:
arma = tsa.ARMA(time_series, order=(3, 3))
arma_result = arma.fit()
prediction = arma_result.predict(start=3)

plt.figure(figsize=(12, 4))
plt.plot(time_series, '-o', label='true')
plt.plot(prediction, '-o', label='model')
plt.legend();


In [ ]:
print('MAE = {0:.3f}'.format(mean_absolute_error(time_series[3:], prediction)))